In [4]:
import pandas as pd
import wrds

In [ ]:
conn = wrds.Connection(wrds_username='yjzhu')
conn.create_pgpass_file() 

Loading library list...
Done


In [169]:
import numpy as np
from scipy.stats.mstats import winsorize
from scipy import stats
import statsmodels.formula.api as sm
import patsy
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error

## Question1

In [170]:
#Sample Selection
funda = conn.raw_sql("""select exchg, gvkey, fyear,datadate, oiadp,ni, csho, at,act,che,lct,dlc,txp,dp,prcc_f,sich,dvt,lt,bkvlps, epsfx,mkvalt         
                         from compa.funda 
                         where consol='C' and indfmt='INDL' and datafmt='STD' and popsrc='D' and  fyear<2019 and fyear>2007
                         """) 
company = conn.raw_sql("""select gvkey, sic,conml,spcsrc
                          from compa.company
                          """) 
funda1=pd.merge(funda,company,on=['gvkey'])
funda1['sic1']=np.where(funda1['sich']>0,funda1['sich'],funda1['sic'])
funda1=funda1.drop(['sich','sic'],axis=1)
funda1['sic1']=funda1['sic1'].astype(str)

In [171]:
funda1

,exchg,gvkey,fyear,datadate,oiadp,ni,csho,at,act,che,...,dp,prcc_f,dvt,lt,bkvlps,epsfx,mkvalt,conml,spcsrc,sic1
0,11.0,001004,2008.0,2009-05-31,125.529,78.651,38.884,1377.511,851.312,112.505,...,40.551,14.7000,0.000,720.616,16.8937,1.92,571.5948,AAR Corp,B,5080.0
1,11.0,001004,2009.0,2010-05-31,95.415,44.628,39.484,1501.042,863.429,79.370,...,38.930,19.7000,0.000,754.692,18.9167,1.16,777.8348,AAR Corp,B,5080.0
2,11.0,001004,2010.0,2011-05-31,137.016,69.826,39.781,1703.727,913.985,57.433,...,59.296,26.3900,2.983,868.438,21.0112,1.81,1049.8206,AAR Corp,B,5080.0
3,11.0,001004,2011.0,2012-05-31,142.360,67.723,40.273,2195.653,1063.272,67.720,...,80.333,12.0500,12.081,1329.631,21.4697,1.65,485.2897,AAR Corp,B,5080.0
4,11.0,001004,2012.0,2013-05-31,136.600,55.000,39.382,2136.900,1033.700,75.300,...,108.600,20.0600,11.900,1217.400,23.3254,1.38,790.0029,AAR Corp,B,5080.0
5,11.0,001004,2013.0,2014-05-31,142.600,72.900,39.560,2199.500,1116.900,89.200,...,113.400,24.3000,11.800,1198.800,25.2654,1.83,961.3080,AAR Corp,B,5080.0
6,11.0,001004,2014.0,2015-05-31,-8.600,10.200,35.423,1515.000,954.100,54.700,...,92.300,29.5400,11.900,669.900,23.8574,-1.40,1046.3954,AAR Corp,B,5080.0
7,11.0,001004,2015.0,2016-05-31,66.100,47.700,34.515,1442.100,873.100,31.200,...,70.800,24.4100,10.400,576.300,25.0847,1.16,842.5112,AAR Corp,B,5080.0
8,11.0,001004,2016.0,2017-05-31,77.200,56.500,34.354,1504.100,888.500,10.300,...,71.000,34.9400,10.200,589.900,26.6112,1.45,1200.3288,AAR Corp,B,5080.0
9,11.0,001004,2017.0,2018-05-31,86.000,15.600,34.716,1524.700,942.700,41.600,...,40.500,44.6900,10.300,588.400,26.9703,2.11,1551.4580,AAR Corp,B,5080.0


In [172]:
#Generate Lag Variable
funda1_lag1 = funda1[['gvkey','fyear','at','act','che','lct','dlc','txp']].copy()
funda1_lag1['fyear'] = funda1_lag1['fyear']+1
funda1_lag1=funda1_lag1.rename(columns = {'at':'at_lag1','act':'act_lag1','che':'che_lag1','lct':'lct_lag1','dlc':'dlc_lag1',
                                       'txp':'txp_lag1'})
funda1=pd.merge(funda1, funda1_lag1, how = 'left', on = ['gvkey','fyear'])

In [173]:
#winsorize the variable
varlist  = ['at','at_lag1','act_lag1','che_lag1','lct_lag1','dlc_lag1','txp_lag1','act','che','lct','dlc','txp','oiadp','ni','csho','dp','dvt','lt','bkvlps']
for var in varlist:    
    funda1[var] = funda1[var].replace(np.Inf, np.nan)    
    funda1[var]=np.where(funda1[var].isnull(), np.nan, winsorize(funda1[var], limits=(0.01,0.01)))

In [174]:
#Construct other Variables
funda1['AverageTotalAsset'] = (funda1['at']+funda1['at_lag1'])/2
funda1['Earnings'] = funda1['oiadp']/funda1['AverageTotalAsset']
funda1['chg_ca']=funda1['act']-funda1['act_lag1']
funda1['chg_cash'] = funda1['che']-funda1['che_lag1']
funda1['chg_cl'] =funda1['lct']-funda1['lct_lag1']
funda1['chg_std'] = funda1['dlc']-funda1['dlc_lag1']
funda1['chg_tp'] = funda1['txp']-funda1['txp_lag1']
funda1['Accural_Comp'] = ((funda1['chg_ca']-funda1['chg_cash']) -(funda1['chg_cl']-funda1['chg_std']-funda1['chg_tp'])-funda1['dp'])/funda1['AverageTotalAsset']
funda1['be'] = funda1['at']-funda1['lt']
funda1['CashFlow']=funda1['Earnings']-funda1['Accural_Comp']

In [175]:
fundaclean1 = funda1[['gvkey','conml','spcsrc','fyear','datadate','epsfx','Accural_Comp','at','dvt','be','bkvlps','CashFlow','Earnings','sic1','mkvalt','csho']].copy()

In [176]:
fundaclean1['dvt'] = fundaclean1['dvt'].replace(np.nan, 0)
fundaclean1['DD'] = np.where(fundaclean1['dvt']==0,0,1)
fundaclean1['Neg_E'] = np.where(fundaclean1['epsfx']<0,1,0)
fundaclean1['Neg_E1'] = np.where(fundaclean1['Earnings']<0,1,0)

In [177]:
# Create lead earnings
fundaclean1_lead1= fundaclean1[['gvkey','fyear','Earnings']].copy()
fundaclean1_lead1['fyear'] = fundaclean1_lead1['fyear']-1
fundaclean1_lead1=fundaclean1_lead1.rename(columns = {'Earnings':'Earnings_lead'})
fundaclean1=pd.merge(fundaclean1,fundaclean1_lead1, how = 'left', on = ['gvkey','fyear'])

In [178]:
# Create lead earnings
fundaclean1_lead1= fundaclean1[['gvkey','fyear','epsfx']].copy()
fundaclean1_lead1['fyear'] = fundaclean1_lead1['fyear']-1
fundaclean1_lead1=fundaclean1_lead1.rename(columns = {'epsfx':'Eps_lead'})
fundaclean1=pd.merge(fundaclean1,fundaclean1_lead1, how = 'left', on = ['gvkey','fyear'])

In [179]:
fundaclean1 = fundaclean1.dropna()

In [180]:
fundaclean1['market_to_book'] = fundaclean1['mkvalt']/(fundaclean1['bkvlps']*fundaclean1['csho'])

In [181]:
fundaclean1['market_to_book'] = fundaclean1['market_to_book'].replace(np.Inf, np.nan)   

In [182]:
fundaclean1 = fundaclean1.dropna()

In [183]:
A = ['A+','A','A-']
B = ['B+','B','B-']
fundaclean1['stock_rate'] = np.where(fundaclean1['spcsrc'].isin(A), 'A', 
                             (np.where(fundaclean1['spcsrc'].isin(B),'B','others')))

In [184]:
fundaclean1 = pd.get_dummies(fundaclean1,columns=['stock_rate'])

In [185]:
fundaclean1

,gvkey,conml,spcsrc,fyear,datadate,epsfx,Accural_Comp,at,dvt,be,...,csho,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead,market_to_book,stock_rate_A,stock_rate_B,stock_rate_others
1,001004,AAR Corp,B,2009.0,2010-05-31,1.16,-0.021397,1501.042,0.000,746.350,...,39.484,0,0,0,0.085508,1.81,1.041408,0,1,0
2,001004,AAR Corp,B,2010.0,2011-05-31,1.81,-0.041990,1703.727,2.983,835.289,...,39.781,1,0,0,0.073017,1.65,1.255997,0,1,0
3,001004,AAR Corp,B,2011.0,2012-05-31,1.65,0.005253,2195.653,12.081,866.022,...,40.273,1,0,0,0.063058,1.38,0.561256,0,1,0
4,001004,AAR Corp,B,2012.0,2013-05-31,1.38,-0.045235,2136.900,11.900,919.500,...,39.382,1,0,0,0.065769,1.83,0.860007,0,1,0
5,001004,AAR Corp,B,2013.0,2014-05-31,1.83,-0.034084,2199.500,11.800,1000.700,...,39.560,1,0,0,-0.004631,-1.40,0.961790,0,1,0
6,001004,AAR Corp,B,2014.0,2015-05-31,-1.40,-0.124485,1515.000,11.900,845.100,...,35.423,1,1,1,0.044706,1.16,1.238190,0,1,0
7,001004,AAR Corp,B,2015.0,2016-05-31,1.16,-0.068445,1442.100,10.400,865.800,...,34.515,1,0,0,0.052406,1.45,0.973103,0,1,0
8,001004,AAR Corp,B,2016.0,2017-05-31,1.45,-0.026882,1504.100,10.200,914.200,...,34.354,1,0,0,0.056788,2.11,1.312981,0,1,0
22,001013,ADC Telecommunications Inc.,B,2009.0,2009-09-30,-4.78,-0.063775,1343.600,0.000,356.200,...,96.600,0,1,0,0.042511,0.78,2.261756,0,1,0
32,001045,American Airlines Group Inc,B-,2017.0,2017-12-31,3.90,-0.038473,51396.000,198.000,3926.000,...,475.508,1,0,0,0.061602,3.03,6.301778,0,1,0


In [186]:
fundaclean1.drop(fundaclean1[fundaclean1['fyear']==2016].index,inplace = True)

In [187]:
fundaclean1.drop(fundaclean1[fundaclean1['fyear']==2017].index,inplace = True)

In [188]:
fundaclean1=fundaclean1.rename(columns={'dvt':'D','bkvlps':'B','epsfx':'E','Accural_Comp':'AC','at':'A','sic1':'SIC'})

In [189]:
fundaclean1

,gvkey,conml,spcsrc,fyear,datadate,E,AC,A,D,be,...,csho,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead,market_to_book,stock_rate_A,stock_rate_B,stock_rate_others
1,001004,AAR Corp,B,2009.0,2010-05-31,1.16,-0.021397,1501.042,0.000,746.350,...,39.484,0,0,0,0.085508,1.81,1.041408,0,1,0
2,001004,AAR Corp,B,2010.0,2011-05-31,1.81,-0.041990,1703.727,2.983,835.289,...,39.781,1,0,0,0.073017,1.65,1.255997,0,1,0
3,001004,AAR Corp,B,2011.0,2012-05-31,1.65,0.005253,2195.653,12.081,866.022,...,40.273,1,0,0,0.063058,1.38,0.561256,0,1,0
4,001004,AAR Corp,B,2012.0,2013-05-31,1.38,-0.045235,2136.900,11.900,919.500,...,39.382,1,0,0,0.065769,1.83,0.860007,0,1,0
5,001004,AAR Corp,B,2013.0,2014-05-31,1.83,-0.034084,2199.500,11.800,1000.700,...,39.560,1,0,0,-0.004631,-1.40,0.961790,0,1,0
6,001004,AAR Corp,B,2014.0,2015-05-31,-1.40,-0.124485,1515.000,11.900,845.100,...,35.423,1,1,1,0.044706,1.16,1.238190,0,1,0
7,001004,AAR Corp,B,2015.0,2016-05-31,1.16,-0.068445,1442.100,10.400,865.800,...,34.515,1,0,0,0.052406,1.45,0.973103,0,1,0
22,001013,ADC Telecommunications Inc.,B,2009.0,2009-09-30,-4.78,-0.063775,1343.600,0.000,356.200,...,96.600,0,1,0,0.042511,0.78,2.261756,0,1,0
34,001045,American Airlines Group Inc,B-,2009.0,2009-12-31,-4.99,-0.031178,25438.000,0.000,-3489.000,...,332.624,0,1,1,0.012192,-1.41,-0.736941,0,1,0
35,001045,American Airlines Group Inc,B-,2010.0,2010-12-31,-1.41,-0.046273,25088.000,0.000,-3945.000,...,333.450,0,1,0,-0.007716,-5.91,-0.658445,0,1,0


In [190]:
model = ols(formula ='Eps_lead~Neg_E+E+Neg_E*E+B+stock_rate_A+stock_rate_B+stock_rate_others+market_to_book',data = fundaclean1).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Eps_lead   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                 2.548e+04
Date:                Thu, 19 Mar 2020   Prob (F-statistic):               0.00
Time:                        15:37:22   Log-Likelihood:            -1.2543e+05
No. Observations:               22933   AIC:                         2.509e+05
Df Residuals:                   22925   BIC:                         2.509e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -6.2875      0.444    -14.151      0.000      -7.158      -5.417
Neg_E                 9.5576      0.930     10.278      0.000       7.735      11.380
E                    12.1389      0.029    422.197      0.000      12.083      12.195
Neg_E:E             -12.2857      0.034   -365.296      0.000     -12.352     -12.220
B                    -1.0586      0.006   -187.060      0.000      -1.070      -1.047
stock_rate_A         -6.8062      1.041     -6.541      0.000      -8.846      -4.767
stock_rate_B          0.6623      0.547      1.210      0.226      -0.410       1.735
stock_rate_others    -0.1437      0.714     -0.201      0.841      -1.543       1.256
market_to_book       -0.0020      0.006     -0.322      0.748      -0.014       0.010
==============================================================================
Omnibus:                    40767.278   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       3231855711.203
Skew:                         -11.233   Prob(JB):                         0.00
Kurtosis:                    1841.945   Cond. No.                     2.16e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.91e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [191]:
pred = model.predict()
MSE = mean_squared_error(fundaclean1['Eps_lead'],pred)
MSE

3297.57215285433

In [192]:
fundaclean2=funda1[['gvkey','conml','spcsrc','fyear','datadate','epsfx','Accural_Comp','at','dvt','be','bkvlps','CashFlow','Earnings','sic1','mkvalt','csho']].copy()

In [193]:
fundaclean2['dvt'] = fundaclean2['dvt'].replace(np.nan, 0)
fundaclean2['DD'] = np.where(fundaclean2['dvt']==0,0,1)
fundaclean2['Neg_E'] = np.where(fundaclean2['epsfx']<0,1,0)
fundaclean2['Neg_E1'] = np.where(fundaclean2['Earnings']<0,1,0)

In [194]:
# Create lead earnings
fundaclean2_lead1= fundaclean2[['gvkey','fyear','Earnings']].copy()
fundaclean2_lead1['fyear'] = fundaclean2_lead1['fyear']-1
fundaclean2_lead1=fundaclean2_lead1.rename(columns = {'Earnings':'Earnings_lead'})
fundaclean2=pd.merge(fundaclean2,fundaclean2_lead1, how = 'left', on = ['gvkey','fyear'])

In [195]:
# Create lead earnings
fundaclean2_lead1= fundaclean2[['gvkey','fyear','epsfx']].copy()
fundaclean2_lead1['fyear'] = fundaclean2_lead1['fyear']-1
fundaclean2_lead1=fundaclean2_lead1.rename(columns = {'epsfx':'Eps_lead'})
fundaclean2=pd.merge(fundaclean2,fundaclean2_lead1, how = 'left', on = ['gvkey','fyear'])

In [196]:
fundaclean2 = fundaclean2.dropna()

In [197]:
fundaclean2

,gvkey,conml,spcsrc,fyear,datadate,epsfx,Accural_Comp,at,dvt,be,...,CashFlow,Earnings,sic1,mkvalt,csho,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead
1,001004,AAR Corp,B,2009.0,2010-05-31,1.16,-0.021397,1501.042,0.000,746.350,...,0.087691,0.066294,5080.0,777.8348,39.484,0,0,0,0.085508,1.81
2,001004,AAR Corp,B,2010.0,2011-05-31,1.81,-0.041990,1703.727,2.983,835.289,...,0.127497,0.085508,5080.0,1049.8206,39.781,1,0,0,0.073017,1.65
3,001004,AAR Corp,B,2011.0,2012-05-31,1.65,0.005253,2195.653,12.081,866.022,...,0.067764,0.073017,5080.0,485.2897,40.273,1,0,0,0.063058,1.38
4,001004,AAR Corp,B,2012.0,2013-05-31,1.38,-0.045235,2136.900,11.900,919.500,...,0.108292,0.063058,5080.0,790.0029,39.382,1,0,0,0.065769,1.83
5,001004,AAR Corp,B,2013.0,2014-05-31,1.83,-0.034084,2199.500,11.800,1000.700,...,0.099852,0.065769,5080.0,961.3080,39.560,1,0,0,-0.004631,-1.40
6,001004,AAR Corp,B,2014.0,2015-05-31,-1.40,-0.124485,1515.000,11.900,845.100,...,0.119855,-0.004631,5080.0,1046.3954,35.423,1,1,1,0.044706,1.16
7,001004,AAR Corp,B,2015.0,2016-05-31,1.16,-0.068445,1442.100,10.400,865.800,...,0.113151,0.044706,5080.0,842.5112,34.515,1,0,0,0.052406,1.45
8,001004,AAR Corp,B,2016.0,2017-05-31,1.45,-0.026882,1504.100,10.200,914.200,...,0.079289,0.052406,5080.0,1200.3288,34.354,1,0,0,0.056788,2.11
22,001013,ADC Telecommunications Inc.,B,2009.0,2009-09-30,-4.78,-0.063775,1343.600,0.000,356.200,...,0.074986,0.011211,3661.0,805.6440,96.600,0,1,0,0.042511,0.78
32,001045,American Airlines Group Inc,B-,2017.0,2017-12-31,3.90,-0.038473,51396.000,198.000,3926.000,...,0.131820,0.093348,4512.0,24740.6812,475.508,1,0,0,0.061602,3.03


In [198]:
fundaclean2['market_to_book'] = fundaclean2['mkvalt']/(fundaclean2['bkvlps']*fundaclean2['csho'])

In [199]:
fundaclean2['market_to_book'] = fundaclean2['market_to_book'].replace(np.Inf, np.nan) 

In [200]:
fundaclean2 = fundaclean2.dropna()

In [201]:
A = ['A+','A','A-']
B = ['B+','B','B-']
fundaclean2['stock_rate'] = np.where(fundaclean2['spcsrc'].isin(A), 'A', 
                             (np.where(fundaclean2['spcsrc'].isin(B),'B','others')))

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [202]:
fundaclean2 = pd.get_dummies(fundaclean2,columns=['stock_rate'])

In [203]:
funda2 = fundaclean2[fundaclean2['fyear'] == 2017]

In [204]:
funda2 = funda2.dropna()

In [205]:
funda2=funda2.rename(columns={'dvt':'D','bkvlps':'B','epsfx':'E','Accural_Comp':'AC','at':'A','sic1':'SIC'})

In [206]:
funda2

,gvkey,conml,spcsrc,fyear,datadate,E,AC,A,D,be,...,csho,DD,Neg_E,Neg_E1,Earnings_lead,Eps_lead,market_to_book,stock_rate_A,stock_rate_B,stock_rate_others
32,001045,American Airlines Group Inc,B-,2017.0,2017-12-31,3.90,-0.038473,51396.000,198.000,3926.000,...,475.508,1,0,0,0.061602,3.03,6.301778,0,1,0
52,001050,CECO Environmental Corp.,C,2017.0,2017-12-31,-0.09,0.002510,438.549,7.792,186.569,...,34.570,1,1,0,0.033838,-0.21,0.950563,0,0,1
74,001072,AVX Corp.,B-,2017.0,2018-03-31,0.03,0.004478,2672.766,75.728,2243.443,...,168.434,1,0,0,0.114501,1.61,1.242548,0,1,0
85,001075,Pinnacle West Capital Corp,A,2017.0,2017-12-31,4.35,-0.022092,17019.082,301.492,5135.730,...,111.752,1,0,0,0.044614,4.54,1.901263,1,0,0
107,001078,Abbott Laboratories,B,2017.0,2017-12-31,0.20,-0.050064,76250.000,1947.000,31098.000,...,1743.602,1,0,0,0.059739,1.31,3.220618,0,1,0
121,001084,Worlds Inc,C,2017.0,2017-12-31,-0.01,-1.783394,0.184,0.000,-4.951,...,246.773,0,1,1,-1.435732,0.03,-2.139296,0,0,1
132,001094,Aceto Corp,D,2017.0,2017-06-30,0.35,-0.008312,1038.185,7.895,405.067,...,30.094,1,0,0,-0.019042,-8.98,1.147837,0,0,1
150,001096,Morguard Corp,B+,2017.0,2017-12-31,26.10,-0.006859,10111.986,7.127,4299.807,...,11.842,1,0,0,0.042584,27.96,0.689467,0,1,0
172,001104,Acme United Corp,A-,2017.0,2017-12-31,1.09,0.022602,114.730,1.447,49.938,...,3.374,1,0,0,0.066510,1.30,1.580996,1,0,0
184,001117,BK Technologies Corp,B-,2017.0,2017-12-31,-0.27,-0.072553,42.801,2.064,32.007,...,13.652,1,1,1,0.059152,-0.01,1.514182,0,1,0


In [207]:
model1= ols(formula = 'Eps_lead~Neg_E+E+Neg_E*E+B+stock_rate_A+stock_rate_B+stock_rate_others+market_to_book', data = funda2).fit()

In [208]:
funda2['predicted_Eps'] = model1.predict()

In [209]:
funda2.to_csv(r"C:/Users/admin/Desktop/Q3_predictions.csv")